In [1]:
# Wonsang Yun, Dept. of mathmetics, Yonsei Univ.
# DEEP LEARNING (STA3140.01-00) Final project 2024/05/07~

import pandas as pd
import os
import shutil
import numpy as np

c:\users\admin\appdata\local\programs\python\python311\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Define Feature
# label 1: containing (both) 'T', 'U' but not containing 'S', 'I', 'O', 'M'
# label 2: containing (both) 'S', 'I' but not containing 'T', 'U', 'O', 'M'
# label 3: containing (both) 'O', 'M' but not containing 'T', 'U', 'S', 'I'
# Then we can gather over 6000 images for training each label.

FEATURE = ['T', 'U', 'S', 'I', 'O', 'M']

# Gather Train, Test, Validation dataset(satisfying above condition) for training.
# Since Train, Test, Validation of this original dataset has same distribution, we just gather this data and then split train and validation.
def gatherImg():

    folder = ['train', 'test', 'validation']

    for j in folder:
        # Gather in train, test, validation(It means not our training set but original training set)
        original_path = f'{j}_v2/{j}/'
        next_path = 'train/'
        df = pd.read_csv(f'written_name_{j}_v2.csv')
        label = df['IDENTITY'].to_numpy()
    
        label1 = []
        label2 = []
        label3 = []
    
        # Find data satisfying above condition
        for i in range(0, len(label)):
            target = label[i]
            # Ignore some label noise
            if type(target) != str:
                continue
            
            # label 1
            if (FEATURE[0] in target) and (FEATURE[1] in target) and (FEATURE[2] not in target) and (FEATURE[3] not in target) and (FEATURE[4] not in target) and (FEATURE[5] not in target):
                label1.append(i)
    
            # label 2
            if (FEATURE[0] not in target) and (FEATURE[1] not in target) and (FEATURE[2] in target) and (FEATURE[3] in target) and (FEATURE[4] not in target) and (FEATURE[5] not in target):
                label2.append(i)
    
            # label 3
            if (FEATURE[0] not in target) and (FEATURE[1] not in target) and (FEATURE[2] not in target) and (FEATURE[3] not in target) and (FEATURE[4] in target) and (FEATURE[5] in target):
                label3.append(i)
        #for(i)
    
        # Transfer
        img_file = df['FILENAME'].to_numpy()
        
        for i in label1:
            file_source = original_path + img_file[i]
            file_destination = next_path +'1/'
            shutil.move(file_source, file_destination)
        #for(i)
            
        for i in label2:
            file_source = original_path + img_file[i]
            file_destination = next_path +'2/'
            shutil.move(file_source, file_destination)
        #for(i)
        
        for i in label3:
            file_source = original_path + img_file[i]
            file_destination = next_path +'3/'
            shutil.move(file_source, file_destination)
        #for(i)
    #for(j)
#END of gatherImg()

In [3]:
gatherImg()

In [3]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transforms

In [5]:
train_path = 'train/'
size = (72, 388)

resize_trans = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor()
])

resize_train = datasets.ImageFolder(root=train_path, transform=resize_trans)

In [4]:
def get_mean_std(dataset):
  meanRGB = [np.mean(image.numpy(), axis=(1,2)) for image,_ in dataset]
  stdRGB = [np.std(image.numpy(), axis=(1,2)) for image,_ in dataset]

  meanR = np.mean([m[0] for m in meanRGB])
  meanG = np.mean([m[1] for m in meanRGB])
  meanB = np.mean([m[2] for m in meanRGB])

  stdR = np.mean([s[0] for s in stdRGB])
  stdG = np.mean([s[1] for s in stdRGB])
  stdB = np.mean([s[2] for s in stdRGB])

  print(meanR, meanG, meanB)
  print(stdR, stdG, stdB)

In [7]:
get_mean_std(resize_train)

0.96593493 0.96593493 0.96593493
0.13249733 0.13249733 0.13249733


In [2]:
# TEST DATASET
# TEST data contains only one feature.
# Gather Train, Test, Validation dataset(satisfying above condition) for testing.
# Since Train, Test, Validation of this original dataset has same distribution, we just gather this data and then split train and validation.

FEATURE = ['T', 'U', 'S', 'I', 'O', 'M']
label_path = ['1/', '1/', '2/', '2/', '3/', '3/']

def gatherTestImg():

    folder = ['train', 'test', 'validation']

    for j in folder:
        # Gather in train, test, validation(It means not our training set but original training set)
        original_path = f'{j}_v2/{j}/'
        next_path = 'test/'
        df = pd.read_csv(f'written_name_{j}_v2.csv')
        label = df['IDENTITY'].to_numpy()
    
        # labelT = []
        # labelU = []
        # labelS = []
        # labelI = []
        # labelO = []
        # labelM = []
        label_idx = [[] for i in range(0, 6)]
    
        # Find data satisfying above condition
        for i in range(0, len(label)):
            target = label[i]
            # Ignore some label noise
            if type(target) != str:
                continue

            # Check condition satisfied
            condition = [(FEATURE[k] in target) for k in range(0, 6)]
            if condition.count(True) == 1:
                idx = condition.index(True)
                label_idx[idx].append(i)
        #for(i)
    
        # Transfer
        img_file = df['FILENAME'].to_numpy()
            
        
        for i in range(0, 6):
            for k in label_idx[i]:
                file_source = original_path + img_file[k]
                file_destination = next_path + FEATURE[i] + '/' + label_path[i]
                shutil.move(file_source, file_destination)
            #for(k)
        #for(i)
        print(label_idx)
    #for(j)
#END of gatherTestImg()

In [37]:
gatherTestImg()

[[0, 94, 178, 195, 314, 406, 437, 463, 484, 530, 534, 688, 765, 781, 870, 912, 922, 928, 979, 1048, 1058, 1068, 1082, 1109, 1153, 1173, 1202, 1204, 1211, 1335, 1337, 1358, 1390, 1486, 1511, 1568, 1624, 1877, 1888, 1939, 2012, 2077, 2100, 2107, 2125, 2181, 2185, 2186, 2196, 2299, 2357, 2383, 2391, 2421, 2510, 2519, 2525, 2585, 2623, 2736, 2743, 2772, 2822, 2863, 2923, 2941, 2953, 2972, 2974, 3024, 3027, 3045, 3086, 3090, 3166, 3185, 3223, 3243, 3244, 3307, 3315, 3319, 3342, 3364, 3414, 3444, 3486, 3499, 3514, 3549, 3660, 3681, 3696, 3840, 3893, 3939, 3973, 4005, 4054, 4143, 4152, 4178, 4210, 4226, 4337, 4338, 4352, 4363, 4413, 4454, 4519, 4524, 4598, 4772, 4860, 4873, 4898, 4901, 4917, 4959, 5038, 5142, 5161, 5162, 5187, 5243, 5309, 5324, 5333, 5357, 5388, 5432, 5446, 5488, 5508, 5560, 5619, 5647, 5656, 5700, 5714, 5755, 5800, 5829, 5846, 5867, 5927, 5928, 6005, 6031, 6039, 6083, 6100, 6105, 6127, 6129, 6133, 6134, 6167, 6201, 6217, 6229, 6271, 6368, 6374, 6398, 6412, 6424, 6435, 6466, 

In [5]:
# TEST Normalization

for i in range(0, 6):
    path = 'test/' + FEATURE[i]
    size = (72, 388)
    
    resize_trans = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor()
    ])
    
    resize_test = datasets.ImageFolder(root=path, transform=resize_trans)
    print(FEATURE[i])
    get_mean_std(resize_test)

T
0.9661202 0.9661202 0.9661202
0.13271476 0.13271476 0.13271476
U
0.9680696 0.9680696 0.9680696
0.12851168 0.12851168 0.12851168
S
0.9660212 0.9660212 0.9660212
0.132027 0.132027 0.132027
I
0.96662706 0.96662706 0.96662706
0.13115089 0.13115089 0.13115089
O
0.96790284 0.96790284 0.96790284
0.12857606 0.12857606 0.12857606
M
0.96551895 0.96551895 0.96551895
0.13397531 0.13397531 0.13397531
